## TCL4 Generator

This notebook aims to implement the simplified TCL4 master equationfor a improved accuracy
from the widely adopted TCL2 method, namely, the Bloch-Redfield equation.

Our open quantum system is defined by the total system-bath Hamiltonian

$$H_T = H_S + H_B + H_I, $$

Where $H_S$ is the isolated system Hamiltonian on the N-dimensional system Hilbert Space
$$H_S = \sum_{n+1}^{N} E_n |n\rangle \langle n|,$$
with nondegenerate energy levels $E_1 < E_2 < ... < E_N.$ $H_B$ is the isolated Hamiltonian of the baths, in the bath Fock space $H_F$. 

$$H_B = \sum_{\alpha, k} \omega_k^{\alpha} b_k^{\alpha^\dagger}b_k^\alpha,$$

$\alpha$ is the label for different baths, $k$ is the label for oscillators with frequency $\omega_k^\alpha > 0$ in bath $\alpha$. $b_k^{\alpha^\dagger}$ and $b_k^{\alpha}$ are the creation and annihilation operators with commutation relations expressed in terms of Kronecker deltas below:
$$[b_k^\alpha, b_q^{\gamma \dagger}] = \delta_{k,q}\delta_{\alpha, \gamma}.$$

$H_I$ is a Hermitian operator describing the interation between the isolated system and the isolated bath, proportional to a dimensionless coupling constant $\lambda$. 

$$H_I = \sum_{\alpha = 1}^{N_b} A^\alpha \otimes F^\alpha$$

$A^{\alpha}$s are Hermitian operators on the system Hilbert space, and $F^{\alpha}$s are operators on corresponding bath Fock space. They are respectively the system coupling operators and the bath coupling operators. Bath coupling operators, $F^{\alpha}$s, are local oscillator displacements expanded as
$$F^{\alpha} = \sum_k g_k^\alpha (b_k^\alpha + b_k^{\alpha \dagger}) $$


In [3]:
import numpy as np
import torch
import random

In [4]:
d0 = 2
d1 = 2
d2 = 2
d3 = 2
d4 = 2
d5 = 2


# 3D Spectral Densities defined as F,C, 
F = np.random.rand(d0, d1, d2, d3, d4, d5)
C = np.random.rand(d0, d1, d2, d3, d4, d5)
R = np.random.rand(d0, d1, d2, d3, d4, d5)

The system and the bath are initialized into a fac- torized state at time t = 0. If the resulting quantum- dynamical map is invertible, then it may be represented by the exact time-convolutionless master equation as follows

$$\frac{d\rho}{dt} = R^0_\rho + R^2(t)\rho + R^4(t)\rho + ...$$

The generators of the dynamics is expressed in terms of tensors $R^{2n}(t)$, each proportional to $\lambda^{2n}.$

$$R_{nm,ij}^{2k}(t) = \delta R_{nm,ij}^{2k}(t) + \delta R_{mn,ji}^{2k\star}(t), k = 0,1,2,...$$

for one term: nm = 11, ij = 11, $\delta_{jm} = \delta_{11} = 1$ 
$$\sum_{a,b,c = 1}^{N} \{A_{na}^{\alpha} * A_{ab}^{\beta} * A_{bc}^{\alpha} * A_{ci}^{\beta}\} [F_{cb,ci,ac}^{\alpha \beta} (t) - R_{cb,ab,bi}^{\alpha \beta} (t)]$$

In [26]:
n = 1
m = 1
i = 1
j = 1
shapeL = (2, 2, 2, 2)
shapeA = (2, 2)
L = np.zeros((2, 2, 2, 2))
A = np.random.rand(2, 2)
# print(A[n,a])

def delta_R (n, m, i, j, F, C, R, L, A):
# Define summation, sum over a, b, and c
    for a in range(2):
        for b in range(2):
            for c in range(2):
                if j == m:
                    L[n,m,i,j] += (A[n,a] * A[a,b] * A[b,c] * A[c,i]
                           * (F[c,b,c,i,a,c]-R[c,b,a,b,b,i]) 
                           + A[n,a] * A[a,b] * A[b,c] * A[c,i] 
                           * R[i,c,a,b,b,i] - A[n,a] * A[a,b] * A[b,c] * A[c,i]
                           * F[b,a,c,i,a,c])
                    print(L)

    for a in range(2):
        for b in range(2):
#             print("A[n,a]", A[n,a])
#             print("A[a,b]", A[n,a])
#             print("R[b,a,n,a,a,i]", R[b,a,n,a,a,i])
#             print("F[b,a,b,i,n,b]", F[b,a,b,i,n,b])
            
#             print("L[n,m,i,j]",L[n,m,i,j])
            L[n,m,i,j] += ((A[n,a] * A[a,b] * A[b,i] * A[j,m] * (R[b,a,n,a,a,i] - F[b,a,b,i,n,b])) 
                    + A[n,a] * A[a,b] * A[b,c] * A[c,i] * F[a,n,b,i,n,b]
                    - A[n,a] * A[a,b] * A[b,i] * A[j,m] * R[i,b,n,a,a,i]
                    + A[n,a] * A[a,b] * A[b, i] * A[j,m] * (C[b,a,j,m,a,i] + R[b,a,j,m,a,i])
                    - A[n,a] * A[a,b] * A[b,i] * A[j,m] * (C[i,b,j,m,a,i] + R[i,a,j,b,n,i])
                    - A[n,a] * A[a,i] * A[j,b] * A[b,m] * (C[a,n,j,b,n,i] + R[a,n,j,b,n,i])
                    + A[n,a] * A[a,i] * A[j,b] * A[b,m] * (C[i,a,j,b,n,i] + R[i,a,j,b,n,i]))
#             print(L[n,m,i,j])
#     print(L)
    return(L)
# for i in 4

Here, one could obtain the TCL4 generator (named L here) by specifying their $n$, $m$, $i$, $j$ values

In [27]:
# matrix element specification, i.e.:
n = 1
m = 0
i = 1
j = 1

L = delta_R(n, m, i, j, F, C, R, L, A)
print(L)

[[[[0.         0.        ]
   [0.         0.        ]]

  [[0.         0.        ]
   [0.         0.        ]]]


 [[[0.         0.        ]
   [0.         0.01552321]]

  [[0.         0.        ]
   [0.         0.        ]]]]
